In [ ]:
import numpy as np
import pandas as pd
from numpy_financial import irr

# Carrega o arquivo com dados de velocidade do vento e potência eólica
vento_curva_potencia = np.loadtxt('VentoCurvaPotencia.txt')  # Coluna 1: velocidade do vento; Colunas 2-4: potência eólica

# Carrega os dados de consumo a partir de um arquivo Excel
dados = pd.read_excel('Brava_consumo.xlsx', sheet_name='Carga2010-2037')

# Extrai a demanda horária da rede de 2022 a 2031 (linhas 3 a 8762 e colunas 9 a 28)
Pd = dados.iloc[2:8762, 8:28].values

# -------------------- Entrada de dados do sistema ----------------

# Brava possui 4 Grupos: (G1:Caterpillar - 320 kVA (256 KW), G2:Perkins I -500 kVA, G3:Perkins II – 500 kVA, e G4:Cummins - 400 kVA)

# 7 CENÁRIOS PARA A SIMULAÇÃO - define-se o tipo de aerogerador a usar, a
# potência, a quantidade, potência da turbina hidráulica e volume do
# reservatório
Cen = np.zeros((7, 4))  # Inicialização da matriz de cenários

Cen[0, :] = [150, 2, 2, 1500]
Cen[1, :] = [300, 1, 3, 1500]
Cen[2, :] = [150, 3, 2, 2250]
Cen[3, :] = [225, 2, 4, 2250]
Cen[4, :] = [225, 3, 4, 3400]
Cen[5, :] = [300, 2, 3, 3000]
Cen[6, :] = [300, 3, 3, 4500]

# Legenda das colunas de Cen:
# 1a - potência do aerogerador;
# 2a - quantidade de aerogeradores;
# 3a - especifica a coluna onde vai pegar a potência do aerogerador correspondente: 2 (150 kW), 3 (225 kW), 4 (300 kW);
# 4a - volume do reservatório em m³.

# Inicialização da matriz de resultados
Res = np.zeros((Cen.shape[0], 14))  # Resultados dos n cenários
Res2 = np.zeros((Cen.shape[0], 8))  # Resultados dos n cenários

for k in range(Cen.shape[0]):
    ano = 20  # Anos do projeto
    RS = Cen[k, 3]  # Capacidade do reservatório em m³
    ERmax = RS / 3600 * 9.8 * 110  # Capacidade do reservatório em kWh
    h = 8760  # Quantidade de horas de simulação

    # Inicialização de variáveis
    Er = np.zeros((h, ano))  # Energia do reservatório
    Erj = ERmax / 2  # Energia inicial no reservatório
    Pt = np.zeros((h, ano))
    Pb = np.zeros((h, ano))
    Erej = np.zeros((h, ano))
    Pge = np.zeros((h, ano))
    Pwr = np.zeros((h, ano))

    nb = 0.86  # Eficiência média das bombas
    nt = 0.86  # Eficiência média das turbinas hidráulicas
    nw = 0.97  # Eficiência elétrica da conversão da potência eólica

    Te = Cen[k, 0]  # Potência unitária do aerogerador
    ne = Cen[k, 1]  # Número de aerogeradores
    PDmin = 256 * 0.5  # O grupo eletrogéneo não pode funcionar abaixo de 50% da carga (256 kW é a potência do grupo menor)

    # Potência eólica
    Pwt = ne * nw * vento_curva_potencia[:h, int(Cen[k, 2]) - 1]

    # Potência nominal da turbina hidráulica
    Ptmax = np.mean(Pwt)

    for i in range(ano):  # Para cada ano
        for j in range(h):  # Para cada hora do ano
            # --------------  PRODUÇÃO EÓLICA E BOMBEAMENTO  ----------------------
            if Pwt[j] > Pd[j, i] * 0.35:  # Se o parque produzir mais que o limite suportado pela rede
                Pwr[j, i] = Pd[j, i] * 0.35  # Injeta-se eólica no valor máximo permitido na rede
                # Aproveita-se o excedente eólico para bombear para o reservatório
                if (Pwt[j] - Pwr[j, i]) * nb < (ERmax - Erj):  # Verifica se o reservatório pode absorver o bombeamento
                    Pb[j, i] = Pwt[j] - Pwr[j, i]  # Registra energia gasta no bombeamento
                    Erj += (Pwt[j] - Pwr[j, i]) * nb  # Atualiza energia no reservatório
                else:  # Caso contrário, bombeia parcialmente e rejeita o excedente
                    Pb[j, i] = (ERmax - Erj) / nb
                    Erj = ERmax  # Reservatório cheio
                    Erej[j, i] = (Pwt[j] - Pwr[j, i]) - (ERmax - Erj) / nb  # Energia rejeitada
            else:  # Caso a produção seja inferior ao limite suportado pela rede
                Pwr[j, i] = Pwt[j]  # Toda potência eólica é injetada diretamente na rede



if (Pd[j, i] - Pwr[j][i]) <= Erj * nt:  # Check if reservoir has enough hydraulic energy to complement wind energy
            
    if (Pd[j][i] - Pwr[j][i]) <= Ptmax:  # Check if the hydroturbine has enough power
                Pt[j][i] = Pd.iloc[j][i] - Pwr[j][i]  # Hydraulic turbine injects remaining electricity into the grid
                Erj -= Pt[j][i] / nt  # Reservoir level decreases
    elif (Pd[j][i] - Pwr[j][i]) > Ptmax and (Pd[j][i] - Pwr[j][i]) <= (Ptmax + PDmin):  
                Pge[j][i] = PDmin  # Ensure minimum for diesel generator
                Pt[j][i] = (Pd.iloc[j][i] - Pwr[j][i]) - Pge[j][i]  # Hydraulic turbine complements everything
                Erj -= Pt[j][i] / nt  # Reservoir level decreases
    elif (Pd[j][i] - Pwr[j][i]) > (Ptmax + PDmin):  # Check if diesel and hydraulic can work together
                Pt[j][i] = Ptmax  # Maximum power of the turbine
                Pge[j][i] = Pd[j][i] - Pwr[j][i] - Pt[j][i]  # Diesel generator complements the rest
                Erj -= Ptmax / nt  # Reservoir level decreases


elif (Pd[j][i] - Pwr[j][i]) > Erj * nt:  # Check if reservoir has enough hydraulic energy to complement wind energy
        if Erj * nt > Ptmax and Erj * nt < (Ptmax + PDmin):
            if Ptmax >= PDmin and (Pd[j][i] - Pwr[j][i]) <= Ptmax:  # Ensure turbine stays below its power
                    Pge[j][i] = PDmin  # Ensure minimum for diesel generator
                    Pt[j][i] = (Pd[j][i] - Pwr[j][i]) - PDmin  # Hydraulic turbine complements everything
                    Erj -= Pt[j][i] / nt  # Reservoir level decreases
            else:
                    Pge[j][i] = Pd[j][i] - Pwr[j][i]  # Diesel generator produces everything since it can't produce below 120

        elif Erj * nt > (Ptmax + PDmin):  # Check if diesel and hydraulic can work together
                Pt[j][i] = Ptmax  # Maximum power of the turbine
                Pge[j][i] = Pd[j][i] - Pwr[j][i] - Pt[j][i]  # Diesel generator complements the rest
                Erj -= Ptmax / nt  # Reservoir level decreases
        else:
                Pge[j][i] = Pd[j][i] - Pwr[j][i]  # Only diesel ensures consumption is less than 120 kW

Er[j][i] = Erj  # Store the current value of reservoir energy temporarily in variable Erj


# pW = percentual de eólica na rede
pW = np.sum(Pwr) / np.sum(Pd) * 100
print('Percentual de Energia eolica na rede = %3.2f' % pW)

# pH = percentual de energia hídrica na rede elétrica
pH = np.sum(Pt) / np.sum(Pd) * 100
print('Percentual de Energia hidrica na rede = %3.2f' % pH)

# pER = percentual de penetração total de Renováveis (eólica+hidro)
pER = (np.sum(Pwr + Pt) / np.sum(Pd)) * 100
print('Total de RE = %3.2f' % pER)

# ERt = energia renovável aproveitada (eolica injetada diretamente e hidraulica), kWh
ERt = np.sum(Pwr + Pt)

# pDG = percentual de energia fossil na rede
pDG = np.sum(Pge) / np.sum(Pd) * 100
print('Percentual de Energia fossil na rede = %3.2f' % pDG)

# pdiss = percentual de energia dissipada devido a reservatorio cheio
pdiss = np.sum(Erej) / (np.sum(Pwt) * ano) * 100
print('Percentual de  Energia dissipada devido a reservatorio cheio: %3.2f' % pdiss)

 

Res.append([Te * ne, np.max(Pb), np.max(Pt), RS, pDG, pH, pW, pER, pdiss])

# Conversão dos resultados para uma tabela pandas
columns = ['Wind', 'Pump', 'Hydro', 'm3', 'pDG', 'pHydro', 'pWind', 'pER', 'pdiss', 'Ctot', 'NPV', 'pback', 'IRR', 'LCOE']
results_table = pd.DataFrame(Res, columns=columns)
print(results_table)





Percentual de Energia eolica na rede = nan
Percentual de Energia hidrica na rede = nan
Total de RE = nan
Percentual de Energia fossil na rede = nan
Percentual de  Energia dissipada devido a reservatorio cheio: 64.75


TypeError: 'numpy.ndarray' object is not callable